# Kpop Dashboard Project

## Import libraries

In [ ]:
from dash import Dash, dcc, html, Output, Input
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px

## Exploratory data analysis

### Load dataset

In [ ]:
# Data retrieved from https://www.kaggle.com/datasets/nicolsalayoarias/all-kpop-idols/?select=kpopidolsv3.csv
data = pd.read_csv('kpopidolsv3.csv')
data

### Data cleaning

### Feature engineering

## Data visualisation

### Idol birthday analysis

1. Calendar plot: idol birthday distribution

2. Histogram plot: idol birth year distribution (all / male / female)

3. Histogram plot: idol birth month distribution (all / male / female)

### Debut analysis

1. Calendar plot: debut anniversary distribution

2. Histogram plot: debut anniversary distribution by year

3. Histogram plot: debut anniversary distribution by month

4. Bubble plot: debut age distribution (all / male / female)

5. Bar plot: number of debuted idols per year

6. Bar plot: number of debuted groups per year

### Meta-analysis

1. Pie chart: distribution of groups by gender

2. Bar plot: number of members per group

3. Pie chart: idol's country of origin distribution

## Dashboard

### Dash app setup

In [ ]:
app = Dash(__name__, external_stylesheets=[dbc.themes.VAPOR])

### Dashboard layout and callback

### Run dashboard 

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True)